# RatJ_Day1SD recording info

In [3]:
import numpy as np
import subjects

sess = subjects.sd.ratJday1[0]
print(sess.recinfo)

filename: /data/Clustering/sessions/RatJ/Day1/RatJ_Day1_2019-05-31_03-55-36.xml 
# channels: 75
sampling rate: 30000
lfp Srate (downsampled): 1250


## Metadata for the session

In [32]:
import pandas as pd
from neuropy.core import Animal

animal = Animal(
    name="Jupiter",
    alias="RatJ",
    tag="sd",
    sex="male",
    experimenter="Bapun Giri",
    experiment="Sleep deprivation SD session",
    day='Day1',
    start=pd.Timestamp('2019-05-31 03:55:36'),
    brain_region='dorsal CA1',
    probe='8x8 buzsaki probe',
    track='linear'
)
animal.filename = sess.filePrefix.with_suffix('.animal')
animal.save()


In [33]:
animal.to_dataframe()

,0
name,Jupiter
alias,RatJ
tag,sd
day,Day1
start,2019-05-31 03:55:36
stop,None
sex,male
weight,None
experimenter,Bapun Giri
experiment,Sleep deprivation SD session


## Set probe configuration
Lets create probe configuration for a 128-channel probe

In [ ]:
%matplotlib widget
from neuropy.core import Shank, Probe, ProbeGroup
from neuropy.plotting import plot_probe

shanks = []
channel_groups = sess.recinfo.channel_groups
badchans = sess.recinfo.skipped_channels

#--- cambridge probe -----------
for i in range(8):
    shank = Shank.auto_generate(
        columns=2,
        contacts_per_column=4,
        xpitch=16.5,
        ypitch=15,
        y_shift_per_column=[0,-7.5],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)
    shanks.append(shank)

probe1 = Probe(shanks)

prbgrp = ProbeGroup()
prbgrp.add_probe(probe1)

prbgrp.filename = sess.filePrefix.with_suffix(".probegroup.npy")
# prbgrp.save()
plot_probe(prbgrp)

## Create epochs for your experimental paradigm
A typical experiment involves multiple epochs such pre sleep, running on track and then another sleep epoch.

In [ ]:
sess.paradigm.to_dataframe()

In [ ]:
import pandas as pd
from neuropy.core import Epoch

epochs = pd.DataFrame(
    {
        "start": [0, 11185,15146,15146],
        "stop": [11160,15021,56092,15146+5*3600],
        "label": ["pre", "maze", "post",'sd'],
    }
)

paradigm= Epoch(epochs=epochs)
paradigm.filename = sess.filePrefix.with_suffix(".paradigm.npy")
# paradigm.save()


# Detect epochs
Here we will various types of epochs which typical for hippocampal recordings.

## Artifacts epochs
A typical session will have some artifacts that may negatively influence many analyses. Using a simple zscore measure, we can identify epochs where signal is above some threshold.

In [ ]:
signal = sess.eegfile.get_signal([1])
artifact_epochs = analyses.detect_artifact_epochs(signal, thresh=7)
artifact_epochs.filename = sess.filePrefix.with_suffix(".artifact.npy")
artifact_epochs.save()
plotting.plot_artifact_epochs(artifact_epochs, signal)


## Ripple epochs
To detect ripples one also needs probegroup.

In [ ]:
from neuropy.analyses import oscillations
signal = sess.eegfile.get_signal()
ripple_epochs =oscillations.detect_ripple_epochs(signal, sess.probegroup)
ripple_epochs.filename = sess.filePrefix.with_suffix('.ripple.npy')
ripple_epochs.save()

In [ ]:
signal = sess.eegfile.get_signal(channel_id=[1, 2, 3, 4], t_start=1, t_stop=1.2)
plotting.plot_signal_traces(signal)

# Importing spiketrains from Phy

In [ ]:
from neuropy.io import PhyIO
from neuropy.core import Neurons
from pathlib import Path
import numpy as np

cluster_path = Path(
    "/data/Clustering/SleepDeprivation/RatJ/Day1/spykcirc/clus_combined"
)
chan_grps = sess.recinfo.channel_groups
sub_dirs = sorted([x for x in cluster_path.iterdir() if x.is_dir()])

spk, peak_chans,waveforms,shank_id = [], [],[],[]
for f_ind, folder in enumerate(sub_dirs):
    print(folder)
    phy_data = PhyIO(folder)
    if phy_data.spiketrains is not None:
        spk.extend(phy_data.spiketrains)
        peak_chans.extend(chan_grps[f_ind][phy_data.peak_channels])
        waveforms.extend(phy_data.peak_waveforms)
        shank_id.extend([f_ind]*len(phy_data.spiketrains))



peak_chans = np.array(peak_chans)

neurons = Neurons(
    np.array(spk, dtype=object),
    t_stop=sess.eegfile.duration,
    sampling_rate=phy_data.sampling_rate,
    peak_channels=peak_chans,
    waveforms=np.array(waveforms,dtype='object'),
    shank_ids=np.array(shank_id)
)

# neurons.filename = sess.filePrefix.with_suffix('.neurons')
# neurons.save()

## Estimate neuron type

In [ ]:
%matplotlib widget
from neuropy.utils import neurons_util
neuron_type = neurons_util.estimate_neuron_type(sess.neurons)
sess.neurons.neuron_type = neuron_type
sess.neurons.filename = sess.filePrefix.with_suffix('.neurons')
sess.neurons.save()

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from neuropy.plotting import plot_raster


plt.plot(phy_data.peak_waveforms[0])
plot_raster(neurons,color='jet',add_vert_jitter=True)

# BinnedSpiketrain and Mua objects using Neurons

In [ ]:
mua =sess.neurons.get_mua()
mua.filename = sess.filePrefix.with_suffix(".mua.npy")
mua.save()   


In [ ]:
%matplotlib widget
from neuropy import plotting
smth_mua = sess.mua.get_smoothed(sigma=0.02)
# plotting.plot_mua(smth_mua)

In [ ]:
from neuropy.analyses import detect_pbe_epochs

pbe = detect_pbe_epochs(smth_mua)
pbe.filename = sess.filePrefix.with_suffix('.pbe')
pbe.save()